In [6]:
print("""prime_numbers = [2]\nfor i in range(3, 100):\n is_prime = True\n for j in prime_numbers:\n if i % j == 0:\n is_prime = False\n break\n if is_prime:\n prime_numbers.append(i)\n\nwith open('prime_numbers.txt', 'w') as f:\n for prime in prime_numbers[:15]:\n f.write(str(prime) + '\\n')""")

prime_numbers = [2]
for i in range(3, 100):
 is_prime = True
 for j in prime_numbers:
 if i % j == 0:
 is_prime = False
 break
 if is_prime:
 prime_numbers.append(i)

with open('prime_numbers.txt', 'w') as f:
 for prime in prime_numbers[:15]:
 f.write(str(prime) + '\n')


In [1]:
import os
# garbage_collection_threshold between 0 and 1 in % of mem
# backend native or cudaMallocAsync
# PYTORCH_CUDA_ALLOC_CONF=<option>:<value>,<option2>:<value2>...
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = f"garbage_collection_threshold:0.99"

In [2]:
from flask import Flask, render_template, request, jsonify
import requests
import torch

In [3]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IGNORE_INDEX
from conversation import conv_templates, SeparatorStyle

from transformers import AutoProcessor, PaliGemmaForConditionalGeneration

from llama_cpp import Llama

from PIL import Image
import requests
import torch
import copy
import json
import time
import transformers

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, LlamaF
import bitsandbytes, flash_attn

/home/tobi/miniconda3/envs/assba/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
model_name = "Hermes-2-Theta-Llama-3-8B"
with open(f"../config.json", "r") as config_file:
    config = json.loads(config_file.read().strip())

pretrained = config['models'][model_name]['path']
tokenizer = AutoTokenizer.from_pretrained(pretrained, trust_remote_code=False)
model = AutoModelForSeq2SeqLM.from_pretrained(
    pretrained,
    torch_dtype=torch.float16,
    device_map=config['torch_device_map'],
    load_in_8bit=False,
    load_in_4bit=False,
    use_flash_attention_2=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: Unrecognized configuration class <class 'transformers.models.llama.configuration_llama.LlamaConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM.
Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, GPTSanJapaneseConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, NllbMoeConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SeamlessM4TConfig, SeamlessM4Tv2Config, SwitchTransformersConfig, T5Config, UMT5Config, XLMProphetNetConfig.

In [13]:
device="cuda"

args = {'chat': [{'role': 'System', 'content': 'Hello, I am the system.'}, {'role': 'User', 'content': 'what is the current date?'}], 'model': 'Hermes-2-Theta-Llama-3-8B', 'manual_system_prompt': '', 'use_functions': True, 'use_image': False}

new_chat = []
new_chat.append({'role': 'system', 'content': config['models'][model_name]['functions']})

for entry in args['chat']:
    if entry['role'] == "User":
        new_chat.append({'role': 'user', 'content': entry['content']})
    if entry['role'] == "AI":
        new_chat.append({'role': 'assistant', 'content': entry['content']})

args['chat'] = new_chat

chat_string = ""

for entry in args['chat']:
    chat_string += f"<|im_start|>{entry['role']}\n"
    chat_string += f"{entry['content']}<|im_end|>\n"
chat_string += f"<|im_start|>assistant\n"

print(chat_string, flush=True)

tokens = tokenizer(chat_string, return_tensors="pt").input_ids.to(device)

gen_inputs = {}
args['tokens'] = tokens

<|im_start|>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions. Here are the available tools: <tools> {"type": "function", "function": {"name": "get_web_info", "description": "get_web_info(symbol: str) -> dict - Get web results for a given query.\n\n    Args:\n        query (str): The web search query.\n\n    Returns:\n        dict: A dictionary containing web search results.\n            Keys:\n                - 'website': The first returned website.\n                - 'website_content': The content of the website.", "parameters": {"type": "object", "properties": {"query": {"type": "string"}}, "required": ["query"]}}} {"type": "function", "function": {"name": "run_python", "description": "run_python(code: str) -> dict - Returns stdout and stderr from running the provided coda.\n\n    Args:\

In [19]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    HammingDiversityLogitsProcessor,
    BeamSearchScorer,
)
import torch

# lets run diverse beam search using 6 beams
num_beams = 6
# define decoder start token ids
input_ids = torch.ones((num_beams, 1), device=model.device, dtype=torch.long)
input_ids = input_ids * model.config.bos_token_id


In [18]:
print(model.config)

LlamaConfig {
  "_name_or_path": "/home/tobi/ai/models/text_to_text/Hermes-2-Theta-Llama-3-8B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128003,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.42.0.dev0",
  "use_cache": true,
  "vocab_size": 128256
}



In [34]:
print(model.__dict__['_modules']['model'].embed_tokens.__dict__)

{'training': False, '_parameters': OrderedDict([('weight', Parameter containing:
tensor([[ 0.0015,  0.0056, -0.0028,  ...,  0.0022, -0.0013,  0.0004],
        [-0.0027,  0.0010, -0.0006,  ...,  0.0002, -0.0014, -0.0020],
        [ 0.0019, -0.0134,  0.0003,  ...,  0.0022,  0.0072,  0.0040],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000, -0.0000,  ...,  0.0000,  0.0000, -0.0000],
        [-0.0000, -0.0000,  0.0000,  ...,  0.0000, -0.0000, -0.0000]],
       device='cuda:0', dtype=torch.float16, requires_grad=True))]), '_buffers': OrderedDict(), '_non_persistent_buffers_set': set(), '_backward_pre_hooks': OrderedDict(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_hooks_with_kwargs': OrderedDict(), '_forward_hooks_always_called': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_forward_pre_hooks_with_kwargs': OrderedDict(), '_state_dict_hooks': OrderedDict(), 

In [38]:

# add encoder_outputs to model keyword arguments
model_kwargs = {
    "encoder_outputs": model.__dict__['_modules']['model'].embed_tokens(
        args['tokens'].repeat_interleave(num_beams, dim=0)#, return_dict=True
    )
}

# instantiate beam scorer
beam_scorer = BeamSearchScorer(
    batch_size=1,
    max_length=model.config.max_length,
    num_beams=num_beams,
    device=model.device,
    num_beam_groups=3,
)

# instantiate logits processors
logits_processor = LogitsProcessorList(
    [
        HammingDiversityLogitsProcessor(5.5, num_beams=6, num_beam_groups=3),
        MinLengthLogitsProcessor(5, eos_token_id=model.config.eos_token_id),
    ]
)

outputs = model.group_beam_search(
    input_ids, beam_scorer, logits_processor=logits_processor, **model_kwargs
)

tokenizer.batch_decode(outputs, skip_special_tokens=True)





response = tokenizer.decode(generated_ids[0][args['tokens'].shape[-1]:], skip_special_tokens=True, clean_up_tokenization_space=True)
output_shape = generated_ids[0][args['tokens'].shape[-1]:].shape
returned_content = [response.strip()]
print(returned_content)

AttributeError: 'LlamaForCausalLM' object has no attribute 'group_beam_search'

In [21]:
print(dir(model))

['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_assisted_decoding', '_auto_class', '_autoset_attn_implementation', '_backward_compatibility_gradient_checkpointing', '_backward_hooks', '_backward_pre_hooks', '_beam_sample', '_beam_search', '_buffers', '_call_impl', '_check_and_enable_flash_attn_2', '_check_and_enable_sdpa', '_compiled_call_impl', '_constrained_beam_search', '_contrastive_search', '_convert_head_mask_to_5d', '_copy_lm_head_original_to_resized', '_create_repo', '_dispatch_accelerate_model', '_expand_inputs_for_generation', '_extract_past_from_model_output', '_forward_hooks', 

In [4]:
initial_memory = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0])

In [5]:
print(initial_memory)

434307072


In [6]:
model = Llama(
    model_path="/home/tobi/ai/models/text_to_text/Meta-Llama-3-70B-Instruct-IQ2_S.gguf",
    n_gpu_layers=-1, # Uncomment to use GPU acceleration
    # seed=1337, # Uncomment to set a specific seed
    n_ctx=1024, # Uncomment to increase the context window
)

llama_model_loader: loaded meta data with 26 key-value pairs and 723 tensors from /home/tobi/ai/models/text_to_text/Meta-Llama-3-70B-Instruct-IQ2_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-70B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 80
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 8192
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 28672
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 64
llama_model_loader: - kv

In [7]:
model_memory = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0]) - initial_memory
print(f"Model memory: {model_memory / 1024 / 1024:.2f} MB", model_memory)

Model memory: 22640.00 MB 23739760640


In [8]:
before_forward_memory = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0])

In [13]:
output = model(
      "Q: Name the planets in the solar system? A: ", # Prompt
      max_tokens=32, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=False # Echo the prompt back in the output
) # Generate a completion, can also call create_completion

Llama.generate: prefix-match hit

llama_print_timings:        load time =     614.86 ms
llama_print_timings:      sample time =       7.55 ms /    17 runs   (    0.44 ms per token,  2251.66 tokens per second)
llama_print_timings: prompt eval time =     351.08 ms /    13 tokens (   27.01 ms per token,    37.03 tokens per second)
llama_print_timings:        eval time =     458.97 ms /    16 runs   (   28.69 ms per token,    34.86 tokens per second)
llama_print_timings:       total time =     920.16 ms /    29 tokens


In [10]:
after_forward_memory = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0])
token_memory = after_forward_memory - before_forward_memory
print(f"Memory per token: {token_memory / 1024 / 1024:.2f} MB", token_memory)

Memory per token: 498.00 MB 522190848


In [14]:
print(output)

{'id': 'cmpl-cd9b635a-65d9-4836-b070-0e9a417291e3', 'object': 'text_completion', 'created': 1716111721, 'model': '/home/tobi/ai/models/text_to_text/Meta-Llama-3-70B-Instruct-IQ2_S.gguf', 'choices': [{'text': ' Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus and Neptune', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 14, 'completion_tokens': 17, 'total_tokens': 31}}


In [23]:
print(json.dumps(torch.cuda.memory_stats_as_nested_dict(), indent=4))

{
    "num_alloc_retries": 0,
    "num_ooms": 0,
    "max_split_size": -1,
    "allocation": {
        "all": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        },
        "small_pool": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        },
        "large_pool": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        }
    },
    "segment": {
        "all": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        },
        "small_pool": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        },
        "large_pool": {
            "current": 0,
            "peak": 0,
            "allocated": 0,
            "freed": 0
        }
    },
    "active": {
        "all": {
            "current": 0,
            "peak": 0

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/get_init_text')
def get_init_text():
    returned_content = [
        {'role': 'System', 'content': 'Hello, I am the system.'},
    ]
    return jsonify({'status': 'success', 'returned_content': returned_content})

@app.route('/send_user_msg', methods=['POST'])
def send_user_msg():
    data = request.get_json()
    user = data['user']
    content = data['content']

    url = f"http://127.0.0.1:10000/infer"
    data_to_send = {}
    data_to_send['content'] = content
    data_to_send['use_image'] = False
    response = requests.post(url, json=data_to_send)    
    ai_reply = response.json()['returned_content']
    
    returned_content = [
        {'role': 'User', 'content': content},
        {'role': 'AI', 'content': "\n".join(ai_reply)}
    ]
    print(f"Received text from {user}: {content}")
    return jsonify({'status': 'success', 'returned_content': returned_content})

if __name__ == '__main__':
    app.run(port=14000)